In [2]:
import numpy as np

In [20]:
def power_iteration(A, n_max_steps=100000, threshold=1e-10, normalize=True, x_init:np.ndarray= None):
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix must be square")
    
    # Start with a random vector
    x = np.random.rand(n, 1)
    
    for _ in range(n_max_steps):
        x_new = A @ x
        x_new = x_new / np.linalg.norm(x_new, np.inf)
        if np.linalg.norm(x_new - x) < threshold:
            break
        x = x_new
    
    if normalize:
        x = x / np.linalg.norm(x)
    
    return x

In [21]:
A = np.array([[2, 1], [1, 2]])
u1 = power_iteration(A)
print(u1)


[[0.70710678]
 [0.70710678]]


In [22]:
def get_orthogonal_complement_projection(u):
    u = u.reshape(-1, 1)
    n, _ = u.shape
    return np.eye(n) - u @ u.T / np.linalg.norm(u)**2

In [23]:
def find_eigenvectors(A: np.ndarray, x_init: np.ndarray):
    n, _ = A.shape
    eigenvectors = []
    
    for _ in range(n):
        # Find dominant eigenvector
        ev = power_iteration(A, x_init=x_init)
        
        # Build projection that removes current eigenvector's component
        proj = get_orthogonal_complement_projection(ev)
        
        # Update initial vector for next eigenvector search
        x_init = proj @ x_init
        x_init = x_init / np.linalg.norm(x_init, np.inf)
        
        # Store this eigenvector
        eigenvectors.append(ev)
    
    return eigenvectors

In [24]:
B = np.array([[2.0, 1.0], [1.0, 2.0]])
x_init = np.random.rand(2, 1)
find_eigenvectors(B, x_init)


[array([[0.70710678],
        [0.70710678]]),
 array([[0.70710678],
        [0.70710678]])]

In [25]:
def diagonalize_symmetric_matrix(A: np.ndarray, x_init: np.ndarray):
    """
    Diagonalize a symmetric matrix A using its eigenvectors.

    Returns:
    - U: matrix whose columns are normalized eigenvectors of A
    - D: diagonal matrix of eigenvalues (U.T @ A @ U)
    """
    # Find eigenvectors
    eigenvectors = find_eigenvectors(A, x_init)

    # Stack eigenvectors side by side to form the matrix U
    U = np.hstack(eigenvectors)

    # Normalize each eigenvector (each column) to unit length
    U = U / np.linalg.norm(U, axis=0, ord=2)

    # Compute diagonal form: D = U.T @ A @ U
    D = U.T @ A @ U

    return U, D

In [26]:
A = np.array([[2.0, 1.0],
              [1.0, 2.0]])

x_init = np.random.rand(2, 1)
diagonalize_symmetric_matrix(A, x_init)


(array([[0.70710678, 0.70710678],
        [0.70710678, 0.70710678]]),
 array([[3., 3.],
        [3., 3.]]))